# Training sample (Notebook/Google Colaboratory version)

この Notebook は [Ultralytics](https://github.com/ultralytics/ultralytics) と Google Colaboratory を用いて YOLO11 物体検出モデルを学習するサンプルです。  

Notebook を用いずに学習を行なう場合は、 [`train.py`](./train.py) を用いてください。

また、 [`README.md`](./README.md) を参照し、データセットの配置を行なってください。

このサンプルでは、Google Driveに作業フォルダーとデータセットのzipファイルを次の形で配置したものとして説明をしております。
```
AI_Contest
└── dataset.zip
```

⚠ データセットおよびワークフォルダーの配置が上記と異なる場合は `DATASET_ROOT` 、 `DATASET_ARCHIVE_FILE` を正しいパスへ変更してください

# Step 1 - ライブラリのインストールとGoogleDriveへの接続

In [ ]:
# Install PyTorch (The content might change depending on the runtime).
!pip install torch torchvision
# Install necessary dependencies.
!pip install tqdm ultralytics
# Additional library for notebook.
!pip install ipywidgets

# Mount Google Drive personal space.
# When executing this cell, an access request window will pop-up.
# Please login and grant the necessary permissions.
from google.colab import drive
drive.mount("/content/drive")
%cd /content

In [ ]:
import os
from shutil import rmtree
DRIVE_DIR = "AI_Contest/"
WORK_DRIVE_DIR = os.path.join("/content/drive/MyDrive/", DRIVE_DIR)
DATASET_ARCHIVE_FILE = os.path.join(WORK_DRIVE_DIR, "dataset.zip")
WORK_DIR = "/content"
DATASET_ROOT = os.path.join(WORK_DIR, "dataset")
DATASET_SAVE_DIR = os.path.join(WORK_DIR, "ultralytics_dataset")
TRAINING_SAVE_DIR = os.path.join(WORK_DRIVE_DIR, "outputs")

if os.path.exists(DATASET_SAVE_DIR):
    rmtree(DATASET_SAVE_DIR)

if not os.path.exists(DATASET_ARCHIVE_FILE):
    print(f"[ERROR] Cannot find the dataset {DATASET_ARCHIVE_FILE}")

! cd "{WORK_DIR}" && unzip "{DATASET_ARCHIVE_FILE}"

## Step 1 - データセットの準備

ここでは 配布しているCOCO形式のデータセットを Ultralytics で利用可能な形式へ変換します。  

この手順は次の処理を行ないます。

1. annotation ファイルを変換
2. 画像を再配置
3. 学習用データを学習用と評価用へ分離(val.jsonが存在しない場合のみ)
4. 学習に用いる設定ファイル(YAML)を作成

なお、このスクリプトはCOCO形式のデータセットに学習用(train.json)、評価用(val.json)、テスト用(test.json)のそれぞれに対してUltralytics向けへ変換を行います。  
また、val.jsonが存在しない場合の学習用データと評価用データ分離の比率は `TRAIN_SPLIT` で制御が可能です。

### Step 1.1 - Annotation ファイルの変換

In [ ]:
from ultralytics.data.converter import convert_coco

def prepare_annotations(src_annotations_path: str, dst_path: str):
    """Converts annotations in Ultralytics' format (YOLO)."""
    # cf. https://docs.ultralytics.com/reference/data/converter/#ultralytics.data.converter.convert_coco
    convert_coco(
        labels_dir=src_annotations_path,
        save_dir=dst_path,
        cls91to80=False
    )
    print(f"Annotations saved to {os.path.join(dst_path, 'labels')}")

src_annotations_path = os.path.join(DATASET_ROOT, "annotations")
prepare_annotations(src_annotations_path, DATASET_SAVE_DIR)

### Step 1.2 - 画像ファイルの再配置

In [ ]:
import json
from tqdm import tqdm

LABELS = ("train", "val", "test")

def load_json(json_path: str) -> dict:
    """ Loads a JSON file from disk."""
    with open(json_path, mode="r", encoding="utf-8") as f:
        data = json.load(f)
    return data

def copy_images(
    image_filenames: list,
    src_images_path: str,
    dst_images_path: str
):
    """
    Creates symbolic links of a list of images from a source directory
    into a destination directory.
    """
    for filename in tqdm(image_filenames):
        src_file = os.path.join(src_images_path, filename)
        dst_file = os.path.join(dst_images_path, filename)

        if os.path.isfile(src_file):  # Only copy files, ignore directories.
            # Create symbolic link to reduce disk memory usage.
            os.symlink(src_file, dst_file)

def prepare_images(
    src_annotations_path: str,
    src_images_path: str,
    dst_path: str
) -> list[dict]:
    """
    Organizes images in Ultralytics' format and returns from the annotations
    the list of object categories (classes) to detect.
    """
    categories = []

    # Copy images into correct folder
    for label in LABELS:
        src_annotations_path_with_label = os.path.join(
            src_annotations_path,
            f"{label}.json"
        )

        # Skip non-existant labels
        if not os.path.exists(src_annotations_path_with_label):
            print(f"[WARNING] No {label} data found.")
            continue

        print(f"Processing '{label}' images...")
        dst_images_path = os.path.join(dst_path, "images", label)
        os.makedirs(dst_images_path, exist_ok=True)

        # Get list of images to copy
        json_data = load_json(src_annotations_path_with_label)
        image_filenames = [img["file_name"] for img in json_data["images"]]

        copy_images(image_filenames, src_images_path, dst_images_path)

        # Get detection categories
        if len(categories) == 0:
            categories = json_data.get("categories", [])

    print(f"Images saved to {os.path.join(dst_path, 'images')}")
    return categories

src_images_path = os.path.join(DATASET_ROOT, "images")

categories = prepare_images(src_annotations_path, src_images_path, DATASET_SAVE_DIR)

if len(categories) == 0:
    print("[WARNING] No detection categories (classes) could be loaded from the annotation files.")

### Step 1.3 - 学習用データを学習用と評価用へ分離

In [ ]:
import glob
TRAIN_SPLIT = 0.8

def split_train_to_train_and_val(dst_path: str):
    train_image_path = os.path.join(dst_path, "images", "train")
    val_image_path = os.path.join(dst_path, "images", "val")
    train_label_path = os.path.join(dst_path, "labels", "train")
    val_label_path = os.path.join(dst_path, "labels", "val")
    os.makedirs(val_image_path)
    os.makedirs(val_label_path)
    files = list(glob.glob(os.path.join(train_image_path, "*")))
    for file in files[int(len(files) * TRAIN_SPLIT):]:
        image_file = os.path.basename(file)
        label_file = os.path.splitext(image_file)[0] + ".txt"
        os.rename(os.path.join(train_image_path, image_file), os.path.join(val_image_path, image_file))
        os.rename(os.path.join(train_label_path, label_file), os.path.join(val_label_path, label_file))

# Generate validation dataset if val.json does not exist.
if not os.path.exists(os.path.join(src_annotations_path, "val.json")):
    print("[INFO] Generate validation dataset from train dataset")
    split_train_to_train_and_val(DATASET_SAVE_DIR)

### Step 1.4 - 設定ファイルの作成

In [ ]:
def prepare_configuration_file(categories, dst_path) -> str:
    """Creates Ultralytics' configuration file."""
    # COCO IDs start from 1 but Ultralytics' start from 0.
    dict_categories = {cat["id"] - 1: cat["name"] for cat in categories}

    # Create YAML configuration file.
    yaml_file_path = os.path.join(dst_path, "data.yaml")
    with open(yaml_file_path, mode="w", encoding="utf-8") as f:
        f.write(f"path: {os.path.abspath(dst_path)}  # dataset root dir\n")
        f.write("train: images/train  # train images (relative to 'path')\n")
        f.write("val: images/val  # val images (relative to 'path')\n")
        f.write("test: images/test  # test images (optional)\n\n")

        f.write("# Classes\n")
        f.write("names:\n")
        for cat_id in sorted(dict_categories.keys()):
            f.write(f"    {cat_id}: {dict_categories[cat_id]}\n")

    print(f"Configuration saved to {yaml_file_path}")
    return yaml_file_path

yaml_conf_path = prepare_configuration_file(categories, DATASET_SAVE_DIR)

## Step 2 - 学習

変換したデータセットを用いて YOLO 11 nano　を学習します

In [ ]:
from ultralytics import YOLO
from datetime import datetime
import torch

# Load a pretrained YOLO11 model.
pretrained_model_path = "/content/weights/yolo11n.pt"
model = YOLO(pretrained_model_path)
print(model)


DEVICE = [0,] if torch.cuda.is_available() else "cpu"
N_EPOCHS = 50
IMAGE_SIZE = 640
BATCH_SIZE = 16

# Train the model.
# cf. https://docs.ultralytics.com/modes/train/#train-settings
model.train(
    data=yaml_conf_path,
    project=TRAINING_SAVE_DIR,
    name=datetime.now().strftime("train_%Y-%m-%d_%H-%M-%S"),
    pretrained=True,
    epochs=N_EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE
)

学習が完了すると、学習済みモデル (`outputs/train_YYYY-MM-DD_HH-MM-SS/weights/best.pt`) がGoogle Driveの作業用フォルダーへ保存されます